<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [5]:
import wandb

import numpy as np
import xgboost as xgb

wandb.init(project="visualize-models", name="xgboost")

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ········


wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\91994/.netrc


W&B Run: https://app.wandb.ai/mother-2020/visualize-models/runs/2zvzfzf6

In [6]:
%pwd

'C:\\Users\\91994\\Downloads\\data-science-bowl-2019'

In [7]:
# label need to be 0 to num_class -1
data = np.loadtxt('./dermatology.data', delimiter=',',
        converters={33: lambda x:int(x == '?'), 34: lambda x:int(x) - 1})
sz = data.shape

In [8]:
train = data[:int(sz[0] * 0.7), :]
test = data[int(sz[0] * 0.7):, :]

train_X = train[:, :33]
train_Y = train[:, 34]

test_X = test[:, :33]
test_Y = test[:, 34]

xg_train = xgb.DMatrix(train_X, label=train_Y)
xg_test = xgb.DMatrix(test_X, label=test_Y)

C:\Users\91994\anaconda3\lib\site-packages\xgboost\core.py:383: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


In [9]:
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 6
wandb.config.update(param)

In [10]:
watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 5

In [11]:
%%wandb
# Add the wandb xgboost callback
bst = xgb.train(param, xg_train, num_round, watchlist, callbacks=[wandb.xgboost.wandb_callback()])
# get prediction
pred = bst.predict(xg_test)
error_rate = np.sum(pred != test_Y) / test_Y.shape[0]
print('Test error using softmax = {}'.format(error_rate))

wandb.summary['Error Rate'] = error_rate

[0]	train-merror:0.01172	test-merror:0.12727
[1]	train-merror:0.01562	test-merror:0.12727
[2]	train-merror:0.01172	test-merror:0.10909
[3]	train-merror:0.00781	test-merror:0.08182
[4]	train-merror:0.00781	test-merror:0.09091
Test error using softmax = 0.09090909090909091


In [12]:
import wandb
wandb.init(project="visualize-sklearn")

W&B Run: https://app.wandb.ai/mother-2020/visualize-sklearn/runs/1cyfzpli

In [13]:
# Visualize single plot
wandb.sklearn.plot_confusion_matrix(y_true, y_probas, labels)

NameError: name 'y_true' is not defined

In [ ]:
# Visualize all classifier plots
wandb.sklearn.plot_classifier(clf, X_train, X_test, y_train, y_test, y_pred, y_probas, labels, model_name='SVC', feature_names=None)

# All regression plots
wandb.sklearn.plot_regressor(reg, X_train, X_test, y_train, y_test,  model_name='Ridge')

# All clustering plots
wandb.sklearn.plot_clusterer(kmeans, X_train, cluster_labels, labels=None, model_name='KMeans')

In [14]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
import pandas as pd
import wandb
# wandb.init(anonymous='allow', project="sklearn")
wandb.init(project="visualize-models", name="sklearn")

# Load data
boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = boston.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train model, get predictions
reg = Ridge()
reg.fit(X, y)
y_pred = reg.predict(X_test)

# Visualize model performance
wandb.sklearn.plot_regressor(reg, X_train, X_test, y_train, y_test, 'Ridge')

wandb: 
wandb: Plotting Ridge.
wandb: Logged summary metrics.
wandb: Logged learning curve.
wandb: Logged outlier candidates.
wandb: WARNING wandb uses only the first 1000 datapoints to create the plots.
wandb: Logged residuals.
